In [ ]:
def get_final_mode_for_trip_document(trip):
    if  trip["data"]["algorithm_chosen"] == "sensing":
        sensed_label = max(trip["data"]["sensing"], key=trip["data"]["sensing"].get)
        final_mode = sensed_mode(sensed_label)
    else:
        label_category_labels = trip["data"]["label_assist_confidences"]["mode_confirm"]
        final_label_category_label = max(label_category_labels, key=label_category_labels.get)

        if final_label_category_label not in accepted_labels["mode_confirm"]:
            final_mode = "Other"
        else:
            final_mode = final_label_category_label
    return final_mode

def get_count_errors():
    rel_errors = label_assist_mode_count_variances["Other"]  # mode_count errors
    lower_rel_error = rel_errors["low"]
    upper_rel_error = rel_errors["high"]

def get_la_mode_counts():
    algorithm = "label_assist"
    
    # find all label assist labeled trips
    # find all label assist trips labeled as mode
    # get the variance in count for the mode

    trip_query = db.Stage_analysis_timeseries.find({
        "$and": [ {"metadata.key":"analysis/confirmed_trip"},
                {"data.algorithm_chosen": {"$in": ["label_assist","sensing"]}}]
        })

    # question: what's the best way to view the alternative trips?
    all_counts = {}  # mode {bike: {count: 1, interval: [0,2]}, car: 2, etc}, purpose, replaced mode 

    # for each label type, get the counts for each of the possible label values
    for label_type in LABEL_CATEGORIES:
        all_counts[label_type] = {}

        for t in trip_query:
        # get the mode for each trip and add to the count for that mode
            label_category_labels = t["data"]["label_assist_confidences"][label_type]
            final_label_category_label = max(label_category_labels, key=label_category_labels.get)

            if final_label_category_label not in all_counts[label_type]: all_counts[label_type][final_label_category_label] = 0
            all_counts[label_type][final_label_category_label] += 1

    return all_counts

def get_la_intervals(all_counts, label_categories, accepted_labels, label_assist_mode_count_variances):
    for label_type in label_categories:
        # Get the interval for the "Other" Labels
        # Get a dictionary of mode keys not in accepted labels. Sum up their values. That will be the "Other" group
        other_dict = {k:v for k,v in d.items() if k not in accepted_labels[label_type]}
        other_count = sum[other_dict.values()]

        la_intervals = {} 

        if other_count > 0:
            rel_errors = label_assist_mode_count_variances["Other"]
            lower_rel_error = rel_errors["low"]
            upper_rel_error = rel_errors["high"]
            interval = [other_count*lower_rel_error, other_count*upper_rel_error] #[other_count - 1.96*v, other_count + 1.96*v]
            all_counts[label_type]["Other"] = {"count": other_count, "interval": interval}

        # Get the intervals for the standard labels.
        for final_label in all_counts[label_type]:
            la_intervals[final_label] = {}

            count = all_counts[label_type][final_label]

            if final_label in accepted_labels[label_type]:
                v = label_assist_mode_count_variances[final_label]
            else:
                v = label_assist_mode_count_variances["Other"]
            margin_of_error = 1.96* v

            # maybe upper error and lower error?

            interval = [count - margin_of_error, count + margin_of_error]  # Or an interval is stored in the error rates collection

            la_intervals[label_type][final_label] = {"count": count, "interval": interval}


def get_sens_mode_counts():
    # for each trip with sensed as the final choice, add to running counts
    algorithm = "sensing"

    trip_query = db.Stage_analysis_timeseries.find({
        "$and": [ {"metadata.key":"analysis/confirmed_trip"},{"data.algorithm_chosen": algorithm}]
        })

    all_sens_counts = {}  # bike: {count: 1, interval: [0,2]}, car: {count: 2, etc}

    for t in trip_query:
    # get the mode for each trip and add to the count for that mode
        mode_fractions = t["data"]["sensing"]
        for s in mode_fractions:
            if s not in all_sens_counts: all_sens_counts[s] = 0
            all_sens_counts[s] += mode_fractions[s]

def get_sens_mode_intervals(all_sens_counts):
    for mode in all_sens_counts:
        # get upper and lower error based on FP, FN, and operating system.
        # might need separate counts for ios and android since they have different rates
    print(1)

In [ ]:
'''Error_Rates = db["Error_Rates"]
error_dictionary = 	{ "label_assist_mode_count_variances": {"Walk": 12, "Bike": 10, "e-bike": 10, "Scooter share": 12, "Drove Alone": 5, "Shared Ride":10,
		            "Taxi/Uber/Lyft": 10, "Bus": 12, "Train": 5, "Free Shuttle": 5, "Air": 1, "Other": 20},
	    "sensing_error_rates": {"FP": {"unknown": 0.2, "walking": 0.2}, "FN": {"unknown": 0.2} },
	    "relative_distance_error": {"ios_HAHF" : {"lower": -0.2, "upper": 0.2, "variance": 0.15}, ios_MAHF : {}, android_HAHF : {} }
}


In [ ]:
'''def number_sensed_mode(mode_number):
        if mode_number == 0:
            return "Other"
        elif mode_number == 1: 
            return "Walk"
        elif mode_number == 2:
            return "Bike"
        elif mode_number ==  3: 
            return "Bus"
        elif mode_number ==  4:
            return "Train"
        elif mode_number ==  5: 
            return "Drove Alone"
        elif mode_number ==  6:
            return "Air"
        elif mode_number == 7: 
            return "Train"
        elif mode_number == 8: 
            return "Train"
        elif mode_number == 9:
            return "Train"
'''